In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GroupKFold, KFold
from xgboost import XGBRegressor
from torchinfo import summary

In [2]:
try:
    #Runs better without CuPy as this dataset is very small
    import cupy as cp
    gpu_available = True
    print("CuPy available - GPU acceleration enabled")
except ImportError:
    gpu_available = False
    print("CuPy not available - using CPU arrays")

CuPy not available - using CPU arrays


In [3]:
def load_data_xgboost(data_frame, feature_columns, target_column, segment_column=None, train_ratio=0.8):
    # Split data
    data_split = int(len(data_frame) * train_ratio)
    training_data, test_data = data_frame[:data_split], data_frame[data_split:]
    
    # Prepare features and targets
    X_train = training_data[feature_columns].values.astype(np.float32)
    y_train = training_data[target_column].values.astype(np.float32)
    segments_train = training_data[segment_column].values if segment_column else None
    
    X_test = test_data[feature_columns].values.astype(np.float32)
    y_test = test_data[target_column].values.astype(np.float32)
    segments_test = test_data[segment_column].values if segment_column else None
    
    print(f"\n{'='*60}")
    print("XGBOOST DATA PREPARATION")
    print(f"{'='*60}")
    print(f"Features: {feature_columns}")
    print(f"Target: {target_column}")
    print(f"Train shape: {X_train.shape}")
    print(f"Test shape: {X_test.shape}")
    
    if segment_column:
        print(f"Number of unique segments (train): {len(np.unique(segments_train))}")
        print(f"Number of unique segments (test): {len(np.unique(segments_test))}")
    
    # Convert to GPU arrays if available
    if gpu_available:
        X_train_gpu = cp.array(X_train)
        y_train_gpu = cp.array(y_train)
        X_test_gpu = cp.array(X_test)
        print("Data converted to GPU arrays")
        return X_train_gpu, y_train_gpu, X_test_gpu, y_test, segments_train, segments_test, test_data
    else:
        return X_train, y_train, X_test, y_test, segments_train, segments_test, test_data

In [4]:
class GPUGridSearchCV:
    #Change verbose to 0 for no output for hyperparameter search and 1 for output
    def __init__(self, base_params, param_grid, cv, scoring='neg_mean_absolute_error', verbose=0):
        self.base_params = base_params
        self.param_grid = param_grid
        self.cv = cv
        self.scoring = scoring
        self.verbose = verbose
        self.best_params_ = None
        self.best_score_ = float('-inf')
        self.best_estimator_ = None
        
    def _create_model(self, params):
        """Create model with consistent device configuration"""
        model_params = {**self.base_params, **params}
        return XGBRegressor(**model_params)
        
    def fit(self, X, y, groups=None):
        keys = list(self.param_grid.keys())
        values = list(self.param_grid.values())
        param_combinations = [dict(zip(keys, v)) for v in product(*values)]
        
        print(f"\n{'='*60}")
        print(f"Testing {len(param_combinations)} parameter combinations...")
        print(f"{'='*60}")
        
        best_score = float('-inf')
        best_params = None
        
        for i, params in enumerate(param_combinations):
            if self.verbose:
                print(f"\n[{i+1}/{len(param_combinations)}] Testing: {params}")
            
            # Create model with current parameters
            cv_model = self._create_model(params)
            
            # Perform cross-validation
            cv_scores = []
            for fold, (train_idx, val_idx) in enumerate(self.cv.split(X, y, groups=groups)):
                try:
                    if gpu_available:
                        # Use GPU arrays for training and validation
                        X_cv_train = X[train_idx]
                        y_cv_train = y[train_idx]
                        X_cv_val = X[val_idx]
                        y_cv_val = y[val_idx]
                        
                        # Train on GPU
                        cv_model.fit(X_cv_train, y_cv_train)
                        pred = cv_model.predict(X_cv_val)
                        
                        # Convert to CPU for evaluation
                        y_cv_val_cpu = cp.asnumpy(y_cv_val)
                        pred_cpu = cp.asnumpy(pred) if hasattr(pred, 'device') else pred
                        
                    else:
                        # Use CPU arrays but GPU XGBoost
                        X_cv_train = X[train_idx]
                        y_cv_train = y[train_idx]
                        X_cv_val = X[val_idx]
                        y_cv_val = y[val_idx]
                        
                        # Train on GPU (XGBoost handles CPU->GPU transfer internally)
                        cv_model.fit(X_cv_train, y_cv_train)
                        pred_cpu = cv_model.predict(X_cv_val)
                        y_cv_val_cpu = y_cv_val
                    
                    # Calculate score
                    score = -mean_absolute_error(y_cv_val_cpu, pred_cpu)
                    cv_scores.append(score)
                    
                    if self.verbose:
                        print(f"  Fold {fold+1}: MAE = {-score:.4f}")
                    
                except Exception as e:
                    print(f"  Error in fold {fold}: {e}")
                    cv_scores.append(float('-inf'))
            
            avg_score = np.mean(cv_scores)
            
            if self.verbose:
                print(f"  Average CV score: {avg_score:.4f}")

            
            if avg_score > best_score:
                best_score = avg_score
                best_params = params
            #Uncomment if you use verbose = 1
            #     print(f"  New best score!")
                
        self.best_score_ = best_score
        self.best_params_ = best_params
        
        # Train final model with best parameters
        print(f"\n{'='*60}")
        print("Training final model with best parameters...")
        print(f"{'='*60}")
        self.best_estimator_ = self._create_model(best_params)
        self.best_estimator_.fit(X, y)
        
        print(f"Best parameters: {self.best_params_}")
        print(f"Best CV score: {self.best_score_:.4f}")
        
        return self

In [5]:
def create_xgboost_visualizations(model, predictions, y_test, test_data, feature_columns):
    """
    Create and save XGBoost model visualizations
    """
    cwd = os.getcwd()
    outPath = os.path.join(cwd, "Plots")
    os.makedirs(outPath, exist_ok=True)
    
    # Scatter plot - Actual vs Predicted
    plt.figure(figsize=(10, 8))
    plt.scatter(y_test, predictions, alpha=0.5, s=30, edgecolors='k', linewidth=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', linewidth=3, label='Perfect Prediction')
    plt.xlabel('Actual Earthquake Significance', fontsize=12)
    plt.ylabel('Predicted Earthquake Significance', fontsize=12)
    plt.title(f'XGBoost: Actual vs Predicted (R²={r2_score(y_test, predictions):.3f})', 
              fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(outPath, 'xgb_actual_vs_predicted_scatter.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {os.path.join(outPath, 'xgb_actual_vs_predicted_scatter.png')}")
    
    # Time series plot of actual vs predicted
    plt.figure(figsize=(16, 6))
    
    sample_size = min(200, len(y_test))
    indices = np.arange(sample_size)
    
    plt.plot(indices, y_test[:sample_size], 'b-', label='Actual Significance', 
             linewidth=2, marker='o', markersize=3, alpha=0.7)
    plt.plot(indices, predictions[:sample_size], 'r--', label='Predicted Significance', 
             linewidth=2, marker='s', markersize=3, alpha=0.7)
    
    plt.xlabel('Sample Index', fontsize=12)
    plt.ylabel('Earthquake Significance', fontsize=12)
    plt.title('XGBoost: Actual vs Predicted Earthquake Significance Over Time', 
              fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    
    plt.savefig(os.path.join(outPath, 'xgb_actual_vs_predicted_timeseries.png'), 
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {os.path.join(outPath, 'xgb_actual_vs_predicted_timeseries.png')}")

In [6]:
def train_xgboost_model(data, feature_columns, target_column, segment_column=None, 
                        param_grid=None, base_params=None, cv_splits=3):
    # Load and prepare data
    X_train, y_train, X_test, y_test, segments_train, segments_test, test_data = load_data_xgboost(
        data, feature_columns, target_column, segment_column
    )
    
    # Default base parameters
    if base_params is None:
        base_params = {
            'objective': 'reg:squarederror',
            'tree_method': 'hist', #Try hist and exact
            'device': 'cuda' if gpu_available else 'cpu',
            'random_state': 42
        }
    
    # Parameter search space
    if param_grid is None:
        param_grid = {
            'max_depth': [4, 6, 8],
            'learning_rate': [0.01, 0.05, 0.1],
            'n_estimators': [100, 200, 300],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }
    
    # Create cross-validator
    if segment_column:
        group_cv = GroupKFold(n_splits=cv_splits)
    else:
        group_cv = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    
    # Perform hyperparameter tuning
    print("\n" + "="*60)
    print("STARTING HYPERPARAMETER TUNING")
    print("="*60)
    
    xgb_cv = GPUGridSearchCV(
        base_params=base_params,
        param_grid=param_grid,
        cv=group_cv,
        scoring='neg_mean_absolute_error',
        verbose=0
    )
    
    try:
        xgb_cv.fit(X_train, y_train, groups=segments_train)
        print("\nTraining completed successfully!")
        
    except Exception as e:
        print(f"\nTraining failed: {e}")
    
    # Get the best model and make predictions
    best_model = xgb_cv.best_estimator_
    
    print("\nMaking predictions...")
    if gpu_available:
        X_test_gpu = cp.array(X_test) if not hasattr(X_test, 'device') else X_test
        predictions = best_model.predict(X_test_gpu)
        predictions = cp.asnumpy(predictions) if hasattr(predictions, 'device') else predictions
    else:
        predictions = best_model.predict(X_test)
    
    # Evaluate the model
    print("\n" + "="*60)
    print("XGBOOST MODEL EVALUATION RESULTS")
    print("="*60)
    print(f'MAE: {mean_absolute_error(y_test, predictions):.4f}')
    print(f'MSE: {mean_squared_error(y_test, predictions):.4f}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')
    print(f'R² Score: {r2_score(y_test, predictions):.4f}')
    
    # Create visualizations
    create_xgboost_visualizations(best_model, predictions, y_test, test_data, feature_columns)
    
    return best_model, predictions, y_test

In [7]:
if __name__ == "__main__":
    print("\n" + "="*60)
    print("EARTHQUAKE SIGNIFICANCE PREDICTION WITH XGBOOST")
    print("="*60)
    
    # Load earthquake data
    earthquake_data = pd.read_csv('./datasets/earthquake_1995-2023.csv')
    feature_columns = ['magnitude', 'cdi', 'mmi', 'tsunami', 'dmin', 'gap', 'depth', 'latitude', 'longitude']
    target_column = 'sig'
    
    print(f"\nDataset shape: {earthquake_data.shape}")
    print(f"Features: {feature_columns}")
    print(f"Target: {target_column} (Earthquake Significance)")
    
    # Prepare parameter grid
    xgb_param_grid = {
        'max_depth': [4, 6, 8],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 300],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
    
    # Train XGBoost model
    xgb_model, xgb_predictions, xgb_y_test = train_xgboost_model(
        data=earthquake_data,
        feature_columns=feature_columns,
        target_column=target_column,
        segment_column=None,
        param_grid=xgb_param_grid,
        cv_splits=3
    )

    print("\n" + "="*60)
    print("TRAINING COMPLETE!")
    print("Check the 'Plots' folder for visualizations:")
    print("   - Actual vs Predicted scatter plot")
    print("   - Time series comparison")
    print("="*60)


EARTHQUAKE SIGNIFICANCE PREDICTION WITH XGBOOST

Dataset shape: (1000, 19)
Features: ['magnitude', 'cdi', 'mmi', 'tsunami', 'dmin', 'gap', 'depth', 'latitude', 'longitude']
Target: sig (Earthquake Significance)

XGBOOST DATA PREPARATION
Features: ['magnitude', 'cdi', 'mmi', 'tsunami', 'dmin', 'gap', 'depth', 'latitude', 'longitude']
Target: sig
Train shape: (800, 9)
Test shape: (200, 9)

STARTING HYPERPARAMETER TUNING

Testing 108 parameter combinations...

Training final model with best parameters...
Best parameters: {'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 100, 'subsample': 0.8, 'colsample_bytree': 1.0}
Best CV score: -104.6660

Training completed successfully!

Making predictions...

XGBOOST MODEL EVALUATION RESULTS
MAE: 25.3366
MSE: 2782.9795
RMSE: 52.7540
R² Score: 0.8104
Saved: /home/david/CS6900/EarthquakePredictionModel/Plots/xgb_actual_vs_predicted_scatter.png
Saved: /home/david/CS6900/EarthquakePredictionModel/Plots/xgb_actual_vs_predicted_timeseries.png

TRAI